In [2]:
import sys
path = "../../.."
if path not in sys.path:
    sys.path.insert(0, path)

In [3]:
from data_retrieval import lipade_groundtruth
from data_retrieval.tools.data_loader import getDataLoader
from sklearn.metrics.pairwise import cosine_similarity
from PIL import Image
from tqdm import tqdm
import numpy as np
import torch
import clip

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
corpus = "lipade_groundtruth"
distancePath = "../results/distance/" + corpus + "/"
rawPath = "../results/raw/" + corpus + "/"

In [ ]:
print(clip.available_models())
modelName = 'ViT-L/14'
modelLowerName = "vitl14"

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [5]:
x,_,y = lipade_groundtruth.getDataset(mode="similar")
for i in range(len(x)):
    x[i] = Image.open(x[i])

In [ ]:
model, preprocess = clip.load(modelName, device)

representations = []
with torch.no_grad():
    for i in tqdm(range(len(x)), desc=modelName):
        image = preprocess(x[i]).unsqueeze(0).to(device)
        representations.append(model.encode_image(image).cpu())

ViT-L/14@336px: 100%|██████████| 279/279 [00:07<00:00, 35.62it/s]


In [ ]:
representations = np.array(representations).reshape((len(x),-1))
sim = np.array(cosine_similarity(representations, representations))

distance = 1 - (sim+1)/2
distance -= np.diag(distance)

In [ ]:
np.save(distancePath + "clip_transfer_" + modelLowerName + ".npy", distance)
np.save(rawPath + "clip_transfer_" + modelLowerName + ".npy", representations)